In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Resources/worldbank_all-data.csv')

In [2]:
# Arrays for for loops to generate country-specific .csvs
countries = ['ABW', 'ASM', 'BEL', 'BRB', 'AFG', 'BRN', 'ALB', 'AND', 'AGO',
       'CUW', 'DMA', 'GIN', 'GRD', 'ARG', 'ARM', 'ATG', 'AUT', 'AUS',
       'AZE', 'BDI', 'BFA', 'BGR', 'BHR', 'BHS', 'BLR', 'BLZ', 'BMU',
       'BEN', 'BGD', 'BTN', 'BIH', 'CAN', 'CHI', 'BOL', 'BRA', 'CHN',
       'CPV', 'CUB', 'CYM', 'BWA', 'CAF', 'DZA', 'HRV', 'KHM', 'TCD',
       'VGB', 'CHL', 'COD', 'CIV', 'COG', 'COM', 'COL', 'CYP', 'CZE',
       'DEU', 'CRI', 'DJI', 'DNK', 'ECU', 'ERI', 'FIN', 'FJI', 'FRA',
       'FRO', 'GIB', 'DOM', 'ETH', 'EGY', 'GNQ', 'EST', 'GRC', 'GRL',
       'GAB', 'GHA', 'GEO', 'GMB', 'GNB', 'GTM', 'GUM', 'HTI', 'JPN',
       'GUY', 'NOR', 'HKG', 'XKX', 'HND', 'HUN', 'IMN', 'IDN', 'IND',
       'IRL', 'IRQ', 'IRN', 'ISL', 'ISR', 'JAM', 'JOR', 'ITA', 'PYF',
       'SLV', 'SWZ', 'FSM', 'KAZ', 'KIR', 'KOR', 'LBN', 'LBR', 'LBY',
       'LIE', 'LSO', 'LTU', 'LUX', 'LVA', 'KEN', 'KGZ', 'MAC', 'MCO',
       'KWT', 'LAO', 'MDV', 'MEX', 'MHL', 'MAR', 'MLI', 'MMR', 'MNE',
       'MDA', 'MNP', 'MDG', 'MOZ', 'MWI', 'NAM', 'NCL', 'MKD', 'NER',
       'NGA', 'NIC', 'NLD', 'MLT', 'NPL', 'OMN', 'PRK', 'MNG', 'MRT',
       'MUS', 'KNA', 'LCA', 'MAF', 'MYS', 'NZL', 'PAK', 'ESP', 'LKA',
       'PAN', 'PER', 'PHL', 'PLW', 'SYC', 'PNG', 'POL', 'PRI', 'PRT',
       'QAT', 'PRY', 'RUS', 'SAU', 'SGP', 'SMR', 'ROU', 'RWA', 'SEN',
       'SLB', 'SOM', 'SLE', 'SSD', 'STP', 'SRB', 'SVK', 'SVN', 'SXM',
       'WSM', 'ARE', 'GBR', 'ZAF', 'SDN', 'SUR', 'SWE', 'TCA', 'CHE',
       'TKM', 'TLS', 'TON', 'TTO', 'TUV', 'SYR', 'USA', 'VCT', 'VEN',
       'VIR', 'TGO', 'TJK', 'VNM', 'THA', 'VUT', 'YEM', 'TUN', 'TZA',
       'UGA', 'UKR', 'URY', 'UZB', 'ZMB', 'ZWE']

years = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996','1997', '1998', '1999']

In [ ]:
# Per-country .csv generation loop
for country in countries:

    # Create per-country dataframe
    df1 = df.loc[df['country_code'] == country]

    # Drop unnecessary columns
    df1.drop('country_name', axis=1, inplace=True)
    df1.drop('country_code', axis=1, inplace=True)
    df1.drop('indicator_name', axis=1, inplace=True)

    # Retain index pre-transposing
    df1.set_index('indicator_code', inplace=True)

    # Transpose matrix (rows are now years)
    dfT = df1.T

    # Drop 1960 to 1999
    for year in years:
        dfT = dfT.drop(year, axis=0)

    # Export per-country .csv
    dfT.to_csv('Countries/' + country + '.csv')

In [76]:
for country in countries:
    df = pd.read_csv('Countries/' + country + '.csv')
    df.fillna('BLANK', inplace = True)

    indicators = df.columns

    for indicator in indicators:
        blank_count = (df[indicator].value_counts(dropna=False)).to_dict()
    
        if 'BLANK' in blank_count:
            if blank_count["BLANK"] > 2:
                df.drop(indicator, axis=1, inplace=True)
            else:
                pass
        else:
            pass

    df.replace ('BLANK', np.nan, inplace = True)
    df = df.mean(skipna=True).to_frame().T
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.rename(index={0: country}, inplace=True)
    df.to_csv('Countries_adjusted/' + country + '.csv')

In [77]:
# Obtain all indicator codes from .csv's to merge into one master list
indicator_codes = []

for country in countries:
    df = pd.read_csv('Countries_adjusted/' + country + '.csv')

    indicators = df.columns

    for indicator in indicators:
        if(indicator in indicator_codes):
            pass
        else:
            indicator_codes.append(indicator)

In [85]:
df = pd.DataFrame(columns = indicator_codes)

# for country in countries:
#     df2 = pd.read_csv('Countries_adjusted/' + country + '.csv')
#     df1 = pd.concat([df1, df2], axis = 1)

df1


,Unnamed: 0,FM.AST.DOMS.CN,EG.EGY.PRIM.PP.KD,BX.KLT.DINV.CD.WD,BN.TRF.CURR.CD,BM.GSR.TRAN.ZS,TM.VAL.OTHR.ZS.WT,TM.VAL.MANF.ZS.UN,SP.RUR.TOTL.ZG,SP.POP.DPND.YG,...,SE.SEC.CUAT.UP.FE.ZS,SE.TER.CUAT.ST.FE.ZS,SE.TER.CUAT.BA.FE.ZS,SE.SEC.CUAT.LO.ZS,SE.SEC.CUAT.LO.FE.ZS,SE.PRM.CUAT.ZS,SE.TER.CUAT.ST.ZS,SE.SEC.CUAT.UP.MA.ZS,SE.TER.CUAT.BA.ZS,SE.PRM.CUAT.FE.ZS


In [162]:
# data = []

df = pd.DataFrame(columns = indicator_codes, index = countries)

dict = pd.read_csv('Countries_adjusted/ABW.csv').drop('Unnamed: 0', axis=1).rename(index={0: 'ABW'})
dict
# for key in dict.keys():
#     df.loc[: key] = dict[key]['ABW']

# df

# dict['Unnamed: 0'][0]

# for country in countries:
#     for row in pd.read_csv('Countries_adjusted/' + country + '.csv').to_dict():
#         data.append(row)

# df = pd.DataFrame(data)

# df

,FM.AST.DOMS.CN,EG.EGY.PRIM.PP.KD,BX.KLT.DINV.CD.WD,BN.TRF.CURR.CD,BM.GSR.TRAN.ZS,TM.VAL.OTHR.ZS.WT,TM.VAL.MANF.ZS.UN,SP.RUR.TOTL.ZG,SP.POP.DPND.YG,SP.POP.6569.MA.5Y,...,SP.POP.5559.MA.5Y,SP.POP.1564.TO.ZS,SP.POP.0014.FE.ZS,TX.VAL.MRCH.WL.CD,TM.VAL.MRCH.R5.ZS,TM.UVI.MRCH.XD.WD,SP.POP.7579.MA.5Y,SP.POP.5054.MA.5Y,SP.POP.1564.MA.ZS,SP.POP.0004.MA.5Y
ABW,2.671767e+09,10.577,3.541787e+07,-8.649417e+07,21.734019,40.438489,63.404569,1.165446,29.354584,3.819257,...,6.44591,69.170863,19.131012,1.125688e+08,0.30325,216.658854,1.630233,7.552054,68.933444,6.642387
